In [1]:
import pandas as pd
import re
import json
from bs4 import BeautifulSoup
import requests

In [8]:
calendar_courses = pd.read_csv("ENVR_Courses_Grouped_yr.csv")
calendar_courses = calendar_courses.rename(columns={'Course Code': 'course_code'})
calendar_courses.head(3)

,course_code,Group ID,Rec_ Yr_level
0,BIOL_V 121,NaN,1
1,ENVR_V 100,NaN,1
2,CHEM_V 121,CHEM_OR_1,1


In [ ]:
calendar_courses = calendar_courses[calendar_courses['course_code']]

In [9]:
with open('all_science_courses.json', 'r') as file:
    all_science_courses = json.load(file)

all_science = pd.DataFrame(all_science_courses)
all_science.head(3)

,course_code,course_title,description,prerequisites,corequisites,themes
0,ANAT 392,Gross Anatomy of the Limbs and Trunk,Lectures and laboratory sessions on the human ...,[],[],[]
1,ANAT 448,Directed Studies in Anatomy,Permission of the Head and supervisor required.,[],[],[]
2,ASTR 101,Introduction to the Solar System,"General principles of the celestial sphere, la...",[PHYS 100],[],[]


In [10]:
def format_course_code(code):
    if '_V' not in code:
        return code[:4] + '_V' + code[4:]
    else: 
        return code

all_science['course_code'] = all_science['course_code'].apply(format_course_code)
all_science.head(3)

,course_code,course_title,description,prerequisites,corequisites,themes
0,ANAT_V 392,Gross Anatomy of the Limbs and Trunk,Lectures and laboratory sessions on the human ...,[],[],[]
1,ANAT_V 448,Directed Studies in Anatomy,Permission of the Head and supervisor required.,[],[],[]
2,ASTR_V 101,Introduction to the Solar System,"General principles of the celestial sphere, la...",[PHYS 100],[],[]


In [12]:
ensc_course_info = pd.merge(calendar_courses, all_science, how='left', on= 'course_code')
ensc_course_info.isnull().sum()

course_code       0
Group ID         15
Rec_ Yr_level     0
course_title     13
description      13
prerequisites    13
corequisites     13
themes           13
dtype: int64

In [7]:
new_courses = pd.read_csv("new_courses_descs.csv")
new_courses['prerequisite'] = new_courses['prerequisite'].apply(lambda x: [item.strip() for item in x.split(',')] if pd.notna(x) else x)
new_courses['corequisite'] = new_courses['corequisite'].apply(lambda x: [item.strip() for item in x.split(',')] if pd.notna(x) else x)

new_courses = new_courses.rename(columns={
    'prerequisite': 'prerequisites',
    'corequisite': 'corequisites',
})
new_courses.head(3)

,course name,course code,course description,prerequisites,corequisites,year level
0,ENVR,100,Environmental Sciences 100: (3) Exploring Envi...,NaN,NaN,1
1,ENVR,205,Environmental Sciences 205: (1) Environmental ...,NaN,NaN,2
2,ENVR,240,Environmental Sciences 240: (3) Environmental ...,[MATH 101],"[STAT 200, STAT 201, BIOL 300]",2


In [11]:
pd.set_option('display.max_colwidth', None)

new_courses['course_code'] = new_courses['course name'] + '_V ' + new_courses['course code'].astype(str)
new_courses.head(3)

,course name,course code,course description,prerequisites,corequisites,year level,course_code
0,ENVR,100,"Environmental Sciences 100: (3) Exploring Environmental Science\nIntroduction to topics in Earth Science, Ecology, Human Health and Energy from an Environmental Science perspective. [3-0-0] Y",NaN,NaN,1,ENVR_V 100
1,ENVR,205,"Environmental Sciences 205: (1) Environmental Science in Practice I\nCommunity building, land connections and academic and career planning in environmental science. [1.5-0-0] Y",NaN,NaN,2,ENVR_V 205
2,ENVR,240,"Environmental Sciences 240: (3) Environmental Problem Solving\nPractical and theoretical approaches to tackling complex environmental challenges with emphasis on critical thinking to analyze real-world issues. [3-0-0] Prerequisite: MATH_V 101. Corequisite: One of STAT_V 200, 201, BIOL_V 300. Y",[MATH 101],"[STAT 200, STAT 201, BIOL 300]",2,ENVR_V 240


In [13]:
new_courses['course_title'] = new_courses['course description'].str.extract(r'\)\s*(.*?)\n')
new_courses.head(3)

,course name,course code,course description,prerequisites,corequisites,year level,course_code,course_title
0,ENVR,100,"Environmental Sciences 100: (3) Exploring Environmental Science\nIntroduction to topics in Earth Science, Ecology, Human Health and Energy from an Environmental Science perspective. [3-0-0] Y",NaN,NaN,1,ENVR_V 100,Exploring Environmental Science
1,ENVR,205,"Environmental Sciences 205: (1) Environmental Science in Practice I\nCommunity building, land connections and academic and career planning in environmental science. [1.5-0-0] Y",NaN,NaN,2,ENVR_V 205,Environmental Science in Practice I
2,ENVR,240,"Environmental Sciences 240: (3) Environmental Problem Solving\nPractical and theoretical approaches to tackling complex environmental challenges with emphasis on critical thinking to analyze real-world issues. [3-0-0] Prerequisite: MATH_V 101. Corequisite: One of STAT_V 200, 201, BIOL_V 300. Y",[MATH 101],"[STAT 200, STAT 201, BIOL 300]",2,ENVR_V 240,Environmental Problem Solving


In [16]:
new_courses['description'] = (
    new_courses['course description']
    .str.extract(r'\n(.*)')[0]
    .str.replace(r'\*+', '', regex=True)
    .str.strip()
    .str.replace(r'\s*[YN]\s*$', '', regex=True)
)
new_courses.head()

,course name,course code,course description,prerequisites,corequisites,year level,course_code,course_title,description
0,ENVR,100,"Environmental Sciences 100: (3) Exploring Environmental Science\nIntroduction to topics in Earth Science, Ecology, Human Health and Energy from an Environmental Science perspective. [3-0-0] Y",NaN,NaN,1,ENVR_V 100,Exploring Environmental Science,"Introduction to topics in Earth Science, Ecology, Human Health and Energy from an Environmental Science perspective. [3-0-0]"
1,ENVR,205,"Environmental Sciences 205: (1) Environmental Science in Practice I\nCommunity building, land connections and academic and career planning in environmental science. [1.5-0-0] Y",NaN,NaN,2,ENVR_V 205,Environmental Science in Practice I,"Community building, land connections and academic and career planning in environmental science. [1.5-0-0]"
2,ENVR,240,"Environmental Sciences 240: (3) Environmental Problem Solving\nPractical and theoretical approaches to tackling complex environmental challenges with emphasis on critical thinking to analyze real-world issues. [3-0-0] Prerequisite: MATH_V 101. Corequisite: One of STAT_V 200, 201, BIOL_V 300. Y",[MATH 101],"[STAT 200, STAT 201, BIOL 300]",2,ENVR_V 240,Environmental Problem Solving,"Practical and theoretical approaches to tackling complex environmental challenges with emphasis on critical thinking to analyze real-world issues. [3-0-0] Prerequisite: MATH_V 101. Corequisite: One of STAT_V 200, 201, BIOL_V 300."
3,ENVR,305,"Environmental Sciences 305: (1) Environmental Science in Practice II\nMentoring, incorporating feedback, strengthening land connections and academic and career planning in environmental science. [1.5-0-0] Prerequisite: ENVR_V 205 Y",[ENVR 205],NaN,3,ENVR_V 305,Environmental Science in Practice II,"Mentoring, incorporating feedback, strengthening land connections and academic and career planning in environmental science. [1.5-0-0] Prerequisite: ENVR_V 205"
4,ENVR,350,Environmental Sciences 350: (3) Applied Environmental Ethics \nThe complexities of socio-ecological issues facing practitioners of environmental sciences including ethics and social equity. [3-0-0]\nPrerequisite: 3rd-year class standing. Y,NaN,NaN,3,ENVR_V 350,Applied Environmental Ethics,The complexities of socio-ecological issues facing practitioners of environmental sciences including ethics and social equity. [3-0-0]


In [18]:
new_courses = new_courses[['course_code', 'course_title', 'description', 'prerequisites', 'corequisites']]
new_courses.head()

,course_code,course_title,description,prerequisites,corequisites
0,ENVR_V 100,Exploring Environmental Science,"Introduction to topics in Earth Science, Ecology, Human Health and Energy from an Environmental Science perspective. [3-0-0]",NaN,NaN
1,ENVR_V 205,Environmental Science in Practice I,"Community building, land connections and academic and career planning in environmental science. [1.5-0-0]",NaN,NaN
2,ENVR_V 240,Environmental Problem Solving,"Practical and theoretical approaches to tackling complex environmental challenges with emphasis on critical thinking to analyze real-world issues. [3-0-0] Prerequisite: MATH_V 101. Corequisite: One of STAT_V 200, 201, BIOL_V 300.",[MATH 101],"[STAT 200, STAT 201, BIOL 300]"
3,ENVR_V 305,Environmental Science in Practice II,"Mentoring, incorporating feedback, strengthening land connections and academic and career planning in environmental science. [1.5-0-0] Prerequisite: ENVR_V 205",[ENVR 205],NaN
4,ENVR_V 350,Applied Environmental Ethics,The complexities of socio-ecological issues facing practitioners of environmental sciences including ethics and social equity. [3-0-0],NaN,NaN


In [22]:
new_courses[['prerequisites', 'corequisites']] = new_courses[['prerequisites', 'corequisites']].applymap(
    lambda x: x if isinstance(x, list) else []
)

C:\Users\tanaym\AppData\Local\Temp\ipykernel_28556\1345804180.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  new_courses[['prerequisites', 'corequisites']] = new_courses[['prerequisites', 'corequisites']].applymap(
C:\Users\tanaym\AppData\Local\Temp\ipykernel_28556\1345804180.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_courses[['prerequisites', 'corequisites']] = new_courses[['prerequisites', 'corequisites']].applymap(


In [23]:
list_of_dicts = new_courses.to_dict(orient='records')
list_of_dicts

[{'course_code': 'ENVR_V 100',
  'course_title': 'Exploring Environmental Science',
  'description': 'Introduction to topics in Earth Science, Ecology, Human Health and Energy from an Environmental Science perspective. [3-0-0]',
  'prerequisites': [],
  'corequisites': []},
 {'course_code': 'ENVR_V 205',
  'course_title': 'Environmental Science in Practice I',
  'description': 'Community building, land connections and academic and career planning in environmental science. [1.5-0-0]',
  'prerequisites': [],
  'corequisites': []},
 {'course_code': 'ENVR_V 240',
  'course_title': 'Environmental Problem Solving',
  'description': 'Practical and theoretical approaches to tackling complex environmental challenges with emphasis on critical thinking to analyze real-world issues. [3-0-0] Prerequisite: MATH_V 101. Corequisite: One of STAT_V 200, 201, BIOL_V 300.',
  'prerequisites': ['MATH 101'],
  'corequisites': ['STAT 200', 'STAT 201', 'BIOL 300']},
 {'course_code': 'ENVR_V 305',
  'course_ti

In [24]:
with open('new_courses_info.json', 'w', encoding='utf-8') as f:
    json.dump(list_of_dicts, f, indent=4)